In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection
import pandas as pd
import urllib3
import pymongo


urllib3.disable_warnings()

In [2]:
gis = GIS("https://txdot.maps.arcgis.com", client_id='etCLdRS2UQTqx7zx')
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://txdot.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=etCLdRS2UQTqx7zx&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=s7vN1pAkORFrmExdd82w1fbiuHpp58
Enter code obtained on signing in using SAML: ········
Successfully logged in as: TXDOT_GIS


In [3]:
"""
These scripts are querying the ArcGIS API, by owner and item type. They further
query by authroitative status and if the data are available to the public.
Lastly, the dictionary that is returned by the API is converted into a list of
directories that shows only the information that is required. In this case, the field properties.

"""
# search content by owner and type
items = gis.content.search(query="owner:TPP_GIS", 
                           item_type='Feature *',
                           max_items=200) # for testing 

# query authoritative layers from content search
item_URL = [item.url for item in items 
            if (item.content_status=='org_authoritative' 
                and item.access=='public' 
                and item.url !=None)]

# create FeatureLayerCollection- need to access the data not just the item
layer_URL = [FeatureLayerCollection(lyr).layers for lyr in item_URL]


# create a list of dict. items of the properties 
lyrNames = []
for lyr in layer_URL:
    for lyrname in lyr:
       lyrNames.append(lyrname.properties)

In [4]:
# check to see how many layers you have
len(lyrNames) 

82

In [5]:
"""
Here is a bunch of Pandas stuff. Bascially I took a list of dictonary layer items, with many fields returned by the API
and then converted it so the fields show what layers they are in. I also included the item_id which can be used 
for links to the data on AGO and ODP. The result is a dictonary with the field as the key and the items (including 
arrays of item_id, layer, applications) as the values. **Note that applications comes from a local csv, not the API

"""

# create a dataframe showing the dict. data from the layer properties 
df_rows = []
for data in lyrNames:
    data_row = data.fields
    lyr_name = data.name
    item_id = data.serviceItemId
      
    for row in data_row:
        row['layer'] = lyr_name
        row['item_id'] = item_id
        df_rows.append(row)
        
df = pd.DataFrame(df_rows)

# groupby field name and concatincate all layers into one field 
# then, drop the duplicates, which leaves one row showing all of the
# field information 
df['layer'] =  df.groupby(['name'])['layer'].transform(lambda x : ','.join(x))
df['item_id'] = df.groupby(['name'])['item_id'].transform(lambda x : ','.join(x))
df = df.drop_duplicates(['name','layer'])

# at this point, add the names of all of the applications from a csv
# note that this comes in as a true list (['app1', 'app2']) so no need to split (see below)
app_list = pd.DataFrame(pd.read_csv('data/applications.csv'))['app_name'].tolist()
df['all_applications'] = [app_list for i in df.index]

# the above code creates [foo, bar, awesome],importing into mongo did not 
# like that format, so changed it to a list 

# get the series and turn it into a useable list ['foo', 'bar', 'awesome']
df['layer'] = df['layer'].str.split(',')
df['item_id'] = df['item_id'].str.split(',')


"""
Create a dictionary and then replace pandas df index key with the field name
this can be used later to cross check fields of layers

NOTE that you can't use df.set_index('name') b/c it drops the name key, which
is important to have for queries, cross checks, etc. Therefore, replacing it
via a dict. compensastaion works nicely

"""
df_dict_wrong_id = df.to_dict('index')
df_dict = {str(k).replace(str(k), v['name']) : v for (k,v) in df_dict_wrong_id.items()}

# df_dict


In [6]:
# a test to see if the amount of layers in objectId is the same as the 
# layers going in. The iloc[0] assumes that objectId is at the first location
# of the list

test_objId = df.iloc[0]['layer']
if len(test_objId) != len(layer_URL):
    print(f'HEY!!! SOMETHING BROKE! Layers going in is {len(layer_URL)} and unique layers in the dataframe is {len(test_objId)}')
else:
    print(f'Layers going in is {len(layer_URL)} and unique layers in the dataframe is {len(test_objId)}')

Layers going in is 82 and unique layers in the dataframe is 82


In [7]:
# connect to the local mongo db
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb")

C:\Users\DANDR1-C\Miniconda3\envs\data-dict-test\lib\site-packages\pymongo\compression_support.py:55: UserWarning: Unsupported compressor: disabled
  warnings.warn("Unsupported compressor: %s" % (compressor,))
C:\Users\DANDR1-C\Miniconda3\envs\data-dict-test\lib\site-packages\pymongo\common.py:781: UserWarning: Unknown option gssapiservicename
  warnings.warn(str(exc))


In [8]:
# load data into mongo
# gonig to iterate over the dict by using the keys and then load_one to mongo
# this won't be the most elegant, but the load_many option was not agreeing with my dict

db = client.testdb

collection = db.test_esri_api_collection

# leaving this drop here for testing
if 'test_esri_api_collection' in db.list_collection_names():
    collection.drop()
    collection = db.test_esri_api_collection
    print('Dropped and created test_esri_api_collection!')


# load the dictionary into mongo, couldn't get insert_many to place nice, 
# so wrote insert_one into a for loop

# using the mongo assigned _id objectid 
list_keys = list(df_dict.keys())
for lyr in list_keys:
    df_dict_load = df_dict[str(lyr)]
    mongo_results = collection.insert_one(df_dict_load)
#     print(f"Loaded document: {df_dict[lyr]['name']}", end=" ")
print('loaded data')


Dropped and created test_esri_api_collection!
loaded data
